# CrewAI와 Amazon Bedrock을 이용한 꿈의 여행지 찾기

이 노트북에서는 CrewAI 프레임워크와 Amazon Bedrock을 사용하여 사용자의 선호도에 기반한 꿈의 여행지를 찾을 수 있는 지능형 에이전트를 구축하는 방법을 살펴보겠습니다. 이 에이전트는 대규모 언어 모델(LLM)과 웹 검색 기능을 활용하여 사용자의 설명과 일치하는 목적지를 연구하고 추천합니다.

### 사전 요구사항

시작하기 전에 다음 항목이 설치되어 있는지 확인하세요:

`boto3`와 `botocore` - AWS 서비스와의 상호작용을 위한 라이브러리

`crewai`와 `crewai_tools` - 에이전트 워크플로우 구축을 위한 라이브러리

**아래 셀의 주석을 해제하고 실행한 후 반드시 커널을 재시작하세요**

pip 설치 오류는 무시하셔도 됩니다.

In [ ]:
!pip install boto3==1.34.162 botocore==1.34.162 crewai==0.70.1 crewai_tools==0.12.1 duckduckgo-search==6.3.1 unstructured==0.16.6 PyPDF2==3.0.1 -q 

In [ ]:
!pip install langchain_aws==0.1.17 --force-reinstall -q

#### 주의사항!

진행하기 전에 커널을 재시작해주세요

Crew.ai가 설치하는 langchain 버전은 이전 실습과 호환되지 않습니다. 이전 실습을 실행하려면 Lab 1의 langchain 버전을 강제로 재설치해야 합니다.

#### Crew.ai

bedrock을 사용하기 위해 [__CrewAI__ __LLM__ api](https://docs.crewai.com/how-to/llm-connections#supported-providers)를 사용할 것입니다.

In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

#### 웹 검색 도구 정의

In [ ]:
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### LLM 구성

Amazon Bedrock을 통해 Anthropic의 Claude-3 모델을 LLM으로 사용할 것입니다. CrewAI는 다양한 LLM 제공자와의 상호작용을 위해 내부적으로 LiteLLM을 사용합니다.

In [ ]:
# Configure the LLM
llm = LLM(model="bedrock/anthropic.claude-3-haiku-20240307-v1:0")

### 에이전트 정의

"여행지 연구원" 역할을 하는 에이전트를 생성할 것입니다. 이 에이전트는 사용자의 여행 선호도와 일치하는 목적지를 찾는 역할을 담당합니다.

In [ ]:
# Define the Agent
travel_agent = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations matching user preferences',
    backstory="You are an experienced travel agent specializing in personalized travel recommendations.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[search]  # Tool for online searching
)

### 작업 정의

에이전트가 수행할 작업을 지정해야 합니다. 작업에는 설명, 예상 출력, 그리고 방금 생성한 에이전트가 할당됩니다.

In [ ]:
# Define the Task
task = Task(
    description="Based on the user's travel preferences: {preferences}, research and recommend suitable travel destinations.",
    expected_output="A list of recommended destinations with brief descriptions written in Korean.",
    agent=travel_agent
)

### Crew 생성

crew는 공통의 목표를 달성하기 위해 함께 일하는 에이전트 팀입니다. 이 경우에는 에이전트가 하나뿐이지만, 프레임워크는 확장성을 허용합니다.

In [ ]:
# Create the Crew
crew = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
)

### 워크플로우 실행

이제 사용자의 여행 선호도를 입력으로 하여 crew를 실행할 수 있습니다.

In [ ]:
# User input for travel preferences
user_input = {
    "preferences": "멋진 스노클링과 활기찬 문화를 즐길 수 있는 열대 해변에서 휴가를 보내고 싶어요."
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

#### Crew가 실행되면 CrewAI는 다음을 수행합니다:

• ReAct(Reasoning and Act)를 사용하여 작업을 행동으로 분해하며, 선택적으로 에이전트에 할당된 도구를 사용합니다.

• 이전 단계를 완료하기 위해 Amazon Bedrock에 여러 번 호출합니다.

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)

### 에이전트에 메모리 추가

CrewAI는 에이전트가 과거 상호작용을 기억하고 학습하는 데 도움이 되는 [여러 가지 메모리 유형](https://docs.crewai.com/concepts/memory#implementing-memory-in-your-crew)을 지원합니다. 이 경우에는 Amazon Bedrock의 임베딩 모델을 사용하여 단기 메모리를 활성화할 것입니다.

In [ ]:
# Enabling Memory in the Agent
crew_with_memory = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
    memory=True,  # Enable memory
    embedder={
        "provider": "aws_bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v1",  # Embedding model for memory
            "vector_dimension": 1024
        }
    },
    
)

In [ ]:
# Executing the Crew with Memory
result_with_memory = crew_with_memory.kickoff(inputs=user_input)
Markdown(result_with_memory.raw)